# Evaluation of Classifiers on Data Other than the Bioreactor Samples

This notebook analyzes the accuracy of the SVM classifier on data other than the bioreactor samples.
The analysis makes use of data built by ClassificationData in the tools directory.
This class augments the global namespace to include data used in analysis.

The training data are:
* all genes (not just regulators);
* averaged;
* do not include T1 (since only one observtion)

# TO DO
1. Plot progression for individual samples for Bioreactor
1. Evaluate average of replications for other progressions
1. Examine the contributions to the classification by gene in the samples that don't result in mycobactin being a strong predictor.
1. Do cross correlation studies of genes to check ortogonality.
1. **Check calculation of pooled, whether average log**
1. Structure codes for the analyses conducted. This structure should be reflected in ``DATA_DCT``.
   1. Selection of genes: myco, myco +, all genes
   1. How counts are processed
      1. Reference - T0/biostat, T0/data source, pooled/data source
1. Evaluation of classifiers
   1. Performance on biostat data
   1. Performance on other samples
      1. Time consistency
      1. Replication consistency
      1. Consistency with expectation
1. Analyses for Eliza:
   1. Accuracy with all 11 and the accuracy as you add in more genes (same as what you just sent but with all 11).
   1. Could you test with just the 10 mycobactin genes (Rv2377c-Rv2386c)?
   1. Could you test with just bacterioferritin (Rv3841)? Just Rv2384?
   1. Is there one gene that gives good accuracy
1. compare accuracy with ranom selection of genes
1. Classifier coefficient profile.
   1. Eliza genes - 1-10
   1. Auto selected genes: 5, 10, 20

# Preliminaries

## Imports

In [1]:
import init
from common import constants as cn
from common_python.util import dataframe
from tools.make_classification_data import ClassificationData

import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import cross_val_score
import seaborn as sns

%matplotlib inline

In [3]:
data = ClassificationData()
dct = data.namespace_dct
global_dct = globals()
global_dct.update(dct)
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'init', 'cn', 'dataframe', 'ClassificationData', 'copy', 'plt', 'np', 'pd', 'sklearn', 'PCA', 'svm', 'cross_val_score', 'sns', '_i2', '_i3', 'data', 'dct', 'global_dct'])

## Constants

In [ ]:
FULL_RUN = False

STAGES = list(cn.STATE_NAMES)
STAGES.remove("Normoxia")

## Helper Functions

In [ ]:
def plotSampleData(data_dct, plot_genes=None, selected_sample=None, is_plot=True):
    """
    Plots trinary values of data for the specified genes.
    
Parameters
    ----------
    data_dct: dict
        key: reference type
        value: sample dictionary
            key: sample name
            value: features
    selected_sample: str
        Sample for which plot is done
    plot_genes: list-str
    """
    ref_types = list(data_dct.keys())
    sample_names = list(data_dct[ref_types[0]].keys())
    if plot_genes is None:
        plot_genes = []
        for key, genes in GENE_DCT.items():
            if key != ALL:
                geneLst = list(genes)
                plot_genes.extend(geneLst)
        plot_genes = list(set(plot_genes))
        plot_genes.sort()
    for sample_name in sample_names:
        if (sample_name == selected_sample) or (selected_sample is None):
            for ref_type in ref_types:
                df_X = data_dct[ref_type][sample_name]
                title = "%s, %s" % (sample_name, ref_type)
                TRINARY.plotExpressionLevels(plot_genes, df_X=df_X, title=title, is_plot=is_plot, figsize=(15, 3), is_color_bar=False)

# Tests
plotSampleData(SAMPLE_DCT, is_plot=False)
print("OK!")

In [ ]:
SAMPLE_DCT.keys()

In [ ]:
def plotProgression(sample_name, is_average=False, is_plot=True):
    """
    Does line plots for the sample.
    The plot row indicates how data are prepared. The columns indicate how
    the classifier is construted.
    
    Paramters
    ---------
    sample_name: str
    is_average: bool
        use average of replicas
    """
    # Extract information on the samples and classifiers
    smpl_refs = list(SAMPLE_DCT.keys())
    sample_data = list(SAMPLE_DCT.values())
    descriptor = sample_data.SAMPLE_DESCRIPTOR_DCT[sample_name]
    if is_average:
        sample_dct = SAMPLE_AVG_DCT
        repl_strs = [""]
    else:
        sample_dct = SAMPLE_DCT
        repl_strs = descriptor.rnm
    key_pairs = CLASSIFIER_DCT.keys()
    [clf_refs, groups] = [a for a in zip(*key_pairs)]
    clf_refs = list(set(clf_refs))  # Reference data used to construct classifier
    groups = list(set(groups))  # Gene groups
    # Plot characteristics
    ncol = len(groups)
    nrow = len(clf_refs)
    # Construct plots
    for smpl_ref in smpl_refs:
        _, axes = plt.subplots(nrow, ncol, figsize=(12, 5))
        icol = 0
        irow = 0
        for clf_ref in clf_refs:
            for group in groups:
                ax = axes[irow, icol]
                clf = CLASSIFIER_DCT[(clf_ref, group)]
                df_X = sample_dct[smpl_ref][sample_name]
                if df_X is not None:
                    clf.plotProgression(df_X, repl_strs, descriptor.cnm, ax=ax, is_plot=False)
                    if icol == 0:
                        ax.text(-2, 5.5, clf_ref, fontsize=18)
                    else:
                        ax.set_yticklabels([])
                    if irow == 0:
                        ax.set_xticklabels([])
                        plot_title = group
                    else:
                        plot_title = ""
                    ax.set_title(plot_title, rotation=45)
                    icol += 1
                    if icol >= ncol:
                        icol = 0
                        irow += 1
        plt.suptitle("Sample Reference: " + smpl_ref, fontsize=18)
        if is_plot:
            plt.show()
        else:
            plt.close()
            
#plotProgression("rustad", is_plot=False)
#plotProgression("rustad", is_plot=False, is_average=True)
#print("OK!")

In [ ]:
def reportSamplePredictions(data_dct):
    for name, df_X in data_dct.items():
        print("\n%s" % name)
        df_pred = CLASSIFIER.predict(df_X)
        new_columns = [STAGE_NAMES[c] for c in df_pred.columns]
        df_pred.columns = new_columns
        print(df_pred)

In [ ]:
SAMPLE_DCT[REF_TYPE_POOLED].__dict__.keys()

In [ ]:
def plotConditions(sample_name, fontsize_label=8, is_plot=True):
    """
    Plots conditions for all the expression reference calculations and classifiers.
    
    Parameters
    ----------
    sample_name: str
    nrow: int
    ncol: int
    """
    sample_data = SAMPLE_DCT[REF_TYPE_POOLED][sample_name]
    descriptor = sample_data.SAMPLE_DCT[sample_name]
    if sample_name == "GSE167232":
        ref_types = [REF_TYPE_BIOREACTOR, REF_TYPE_POOLED]
    else:
        ref_types = [REF_TYPE_BIOREACTOR, REF_TYPE_SELF, REF_TYPE_POOLED]
    ref_type_label_dct = {REF_TYPE_BIOREACTOR: "bioreactor T0",
                      REF_TYPE_POOLED: "pooled",
                      REF_TYPE_SELF: "self",
                         }
    ncol = len(CLASSIFIER_DCT.keys())
    nrow = len(ref_types)
    _, axes = plt.subplots(nrow, ncol, figsize=(20, 10))
    icol = 0
    irow = 0
    significance_str = None
    for ref_idx, sample_ref_type in enumerate(ref_types):
        for clf_key, clf in CLASSIFIER_DCT.items():
            ax = axes[irow, icol]
            df_X = SAMPLE_DCT[sample_ref_type][sample_name]
            if df_X is not None:
                significance_str = clf.plotConditions(df_X, descriptor.cnm, state_names=STAGE_NAMES, ax=ax, is_plot=False,
                                   fontsize_label=fontsize_label, shading_offset=0.2,
                                   state_probs=cn.STATE_PROBS, num_exp=24)
                if icol == 0:
                    row_title = ref_type_label_dct[sample_ref_type]
                    ax.set_ylabel(row_title, fontsize=16)
                    # ax.text(-2, 5.5, row_title, fontsize=16)
                else:
                    ax.set_yticklabels([])
                if irow == 0:
                    plot_title = ", ".join(clf_key)
                else:
                    plot_title = ""
                if ref_idx < len(ref_types) - 1:
                    ax.set_xticklabels([])
                ax.set_title(plot_title, rotation=45)
                icol += 1
                if icol >= ncol:
                    icol = 0
                    irow += 1
    suptitle = "%s: %s" % (sample_name, significance_str)
    plt.suptitle(suptitle, fontsize=18)
    if not is_plot:
        plt.close()
    
# Tests
plotConditions("AM_MDM", is_plot=False)
print("OK!")

In [ ]:
CLASSIFIER_DCT.keys()

In [ ]:
df_X = SAMPLE_DCT[REF_TYPE_POOLED]["GSE167232"]
df_X.to_csv("sample.csv")
clf = CLASSIFIER_DCT[("T0", "all")]
descriptor = sample_data.SAMPLE_DESCRIPTOR_DCT["GSE167232"]
clf.plotConditions(df_X, descriptor.cnm, state_names=STAGE_NAMES, is_plot=False,
                                    shading_offset=0.05)

# Gene Expression

**Observations**
1. Prefer control values that avoid having a constant value of the trinary value over the samples (since such a situation is unhelpful for distinguishing classes).
1. On training data (Bioreactor)
   1. Many genes have the same within class values
   1. Few genes have the same between class values

**TO DO**: Plot data in pairs, 1 column, 2 rows; this allows comparisons of the expressions for T0 vs. pooled.

## AM, MDM

In [ ]:
# With Bioreactor reference data
plotSampleData(SAMPLE_DCT, selected_sample="AM_MDM")

## AW

In [ ]:
# With Bioreactor reference data
plotSampleData(SAMPLE_DCT, selected_sample="AW")

## galagan

In [ ]:
# With Bioreactor reference data
plotSampleData(SAMPLE_DCT, selected_sample="galagan")

## rustad

In [ ]:
# With Bioreactor reference data
plotSampleData(SAMPLE_DCT, selected_sample="rustad")

## GSE167232

In [ ]:
# With Bioreactor reference data
plotSampleData(SAMPLE_DCT, selected_sample="GSE167232")

## GSE67035

In [ ]:
# With Bioreactor reference data
plotSampleData(SAMPLE_DCT, selected_sample="GSE67035")

# Condition Analysis

Analyzes data sets with distinct conditions.

**Notes**
1. Calculate the probability of having of different configurations of the classifications if class predictions are $p_c$.

## AW

In [ ]:
plotConditions("AW")

## AM, MDM

In [ ]:
plotConditions("AM_MDM")

## GSE167232

In [ ]:
plotConditions("GSE167232", fontsize_label=7)

## GSE167232R

In [ ]:
plotConditions("GSE167232R", fontsize_label=7)

## GSE67035

In [ ]:
plotConditions("GSE67035", fontsize_label=8)

# Progression Analysis

In [ ]:
#CLASSIFIER.plotFeatureContributions(df_X.loc["TB_AM1",:], class_names=STAGE_NAMES)
if False:
    df_predict = CLASSIFIER.predict(SAMPLE_DATA_DCT["GSE167232"])
    indexArr = np.reshape(list(df_predict.index), (4,3))
    num_row = 4
    num_col =3
    _, axes = plt.subplots(num_row, num_col, figsize=(20, 12))
    for irow in range(4):
        for icol in range(3):
            if icol == 0:
                is_ylabel = True
            else:
                is_ylabel = False
            if icol == num_col - 1:
                is_legend = True
            else:
                is_legend = False
            if irow == num_row - 1:
                is_xlabel = True
            else:
                is_xlabel = False
            instance = indexArr[irow, icol]
            ser_X = SAMPLE_DATA_DCT["GSE167232"].loc[instance, :]
            CLASSIFIER.plotFeatureContributions(ser_X, class_names=STAGE_NAMES, title=instance, ax=axes[irow,icol], is_plot=False,
                                               is_xlabel=is_xlabel, is_ylabel=is_ylabel, is_legend=is_legend)
    plt.show()

## Rustad

In [ ]:
plotProgression("rustad")

In [ ]:
plotProgression("rustad", is_average=True)

## Galagan

In [ ]:
plotProgression("galagan")

In [ ]:
plotProgression("galagan", is_average=True)